### Madina Hayva Putri - 5220411187
### Pemrosesan Teks - B

# Scraping Komentar Youtube Berdasarkan Kata Kunci

In [2]:
!pip install google-api-python-client
!pip install tqdm

In [3]:
from google.colab import userdata
from googleapiclient.discovery import build
import pandas as pd
from tqdm import tqdm

In [4]:
api_key = userdata.get('YouTube')  # ganti nama secret dengan nama yang sama di tab "Rahasia"
youtube = build('youtube', 'v3', developerKey=api_key)

print("Berhasil")

Berhasil


In [5]:
# parameter pencarian video dengan kata kunci
query = "review makanan tiktokers"
max_komen_per_video = 50
min_total_komen = 1000

In [6]:
videos = []
next_page_token = None

print("Mengambil daftar video berdasarkan kata kunci...")
while len(videos) < (min_total_komen // max_komen_per_video) + 5:
    request = youtube.search().list(
        q=query,
        part="id,snippet",
        type="video",
        maxResults=50,
        pageToken=next_page_token
    )
    response = request.execute()
    videos += response["items"]
    next_page_token = response.get("nextPageToken")
    if not next_page_token:
        break

print(f"Ditemukan {len(videos)} video terkait '{query}'\n")


Mengambil daftar video berdasarkan kata kunci...
Ditemukan 50 video terkait 'review makanan tiktokers'



In [7]:
data_komen = []
total_komen = 0

for video in tqdm(videos, desc="Mengambil komentar"):
    video_id = video["id"]["videoId"]
    channel_name = video["snippet"]["channelTitle"]
    video_title = video["snippet"]["title"]

    next_page_token = None
    count = 0

    while count < max_komen_per_video:
        request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            maxResults=100,
            pageToken=next_page_token,
            textFormat="plainText"
        )
        response = request.execute()

        for item in response["items"]:
            comment = item["snippet"]["topLevelComment"]["snippet"]
            data_komen.append({
                "nama_channel": channel_name,
                "judul_video": video_title,
                "author": comment["authorDisplayName"],
                "komentar": comment["textDisplay"]
            })
            count += 1
            total_komen += 1
            if count >= max_komen_per_video:
                break  # hanya berhenti per video

        next_page_token = response.get("nextPageToken")
        if not next_page_token:
            break  # komentar video habis

print(f"\nTotal komentar terkumpul: {len(data_komen)}")

Mengambil komentar: 100%|██████████| 50/50 [00:09<00:00,  5.48it/s]


Total komentar terkumpul: 2152


## Preprocessing

In [8]:
!pip install nltk
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 5.3 MB/s eta 0:00:00


In [9]:
import nltk
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re

In [10]:
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
stop_words = set(stopwords.words("indonesian"))
stemmer = StemmerFactory().create_stemmer()

In [12]:
df = pd.DataFrame(data_komen)

In [13]:
# Casefolding
df["casefolding"] = df["komentar"].str.casefold() #mengubah huruf menjadi kecil

In [14]:
# Stripping
df["stripping"] = df["casefolding"].str.strip()

In [15]:
normalisasi_dict = {
    "gak": "tidak",
    "ga": "tidak",
    "nggak": "tidak",
    "ngga": "tidak",
    "engga": "tidak",
    "enggak": "tidak",
    "tdk": "tidak",
    "tak": "tidak",
    "kagak": "tidak",
    "bgt": "banget",
    "bngt": "banget",
    "bgd": "banget",
    "bet": "banget",
    "amat": "banget",
    "parah": "banget",
    "abis": "banget",
    "abiz": "banget",
    "aj": "saja",
    "aja": "saja",
    "doang": "saja",
    "tok": "saja",
    "doa": "saja",
    "udh": "sudah",
    "sdh": "sudah",
    "dah": "sudah",
    "udah": "sudah",
    "dahh": "sudah",
    "uda": "sudah",
    "ud": "sudah",
    "yg": "yang",
    "yng": "yang",
    "y": "yang",
    "dgn": "dengan",
    "dg": "dengan",
    "sm": "sama",
    "ama": "sama",
    "ma": "sama",
    "sy": "saya",
    "gue": "saya",
    "gua": "saya",
    "aq": "saya",
    "aqi": "saya",
    "aqh": "saya",
    "gws": "semoga cepat sembuh",
    "brb": "sebentar",
    "btw": "ngomong-ngomong",
    "bentar": "sebentar",
    "sebntar": "sebentar",
    "ntar": "nanti",
    "ntr": "nanti",
    "trs": "terus",
    "trus": "terus",
    "tros": "terus",
    "lg": "lagi",
    "lgi": "lagi",
    "lagi2": "lagi-lagi",
    "dl": "dulu",
    "dlu": "dulu",
    "dul": "dulu",
    "td": "tadi",
    "tadi2": "tadi-tadi",
    "klo": "kalau",
    "kl": "kalau",
    "kaloo": "kalau",
    "kalo": "kalau",
    "klw": "kalau",
    "klu": "kalau",
    "gmn": "bagaimana",
    "gmnich": "bagaimana",
    "gmnsi": "bagaimana",
    "gmna": "bagaimana",
    "bsk": "besok",
    "besok2": "besok-besok",
    "bsok": "besok",
    "kmrn": "kemarin",
    "kmrn2": "kemarin-kemarin",
    "kmrin": "kemarin",
    "kmrnku": "kemarinku",
    "knp": "kenapa",
    "kenpa": "kenapa",
    "kenap": "kenapa",
    "kenapaa": "kenapa",
    "tp": "tapi",
    "tpi": "tapi",
    "tpn": "tapi",
    "skr": "sekarang",
    "skrg": "sekarang",
    "skang": "sekarang",
    "skg": "sekarang",
    "skrng": "sekarang",
    "dr": "dari",
    "drpd": "daripada",
    "dri": "dari",
    "dran": "dari",
    "dgini": "begini",
    "bgini": "begini",
    "bgitu": "begitu",
    "gitu": "begitu",
    "gini": "begini",
    "kyk": "seperti",
    "kaya": "seperti",
    "sprti": "seperti",
    "ky": "seperti",
    "spt": "seperti",
    "smngat": "semangat",
    "semngt": "semangat",
    "semgt": "semangat",
    "smangat": "semangat",
    "plis": "tolong",
    "tolongin": "tolong",
    "plz": "tolong",
    "please": "tolong",
    "pliss": "tolong",
    "pls": "tolong",
    "plszz": "tolong",
    "makasi": "terima kasih",
    "mksh": "terima kasih",
    "makasih": "terima kasih",
    "mksih": "terima kasih",
    "thx": "terima kasih",
    "thanks": "terima kasih",
    "trims": "terima kasih",
    "trimakasih": "terima kasih",
    "tq": "terima kasih",
    "cpt": "cepat",
    "cepet": "cepat",
    "cptan": "cepat",
    "cpt2": "cepat-cepat",
    "brasa": "berasa",   # tambahan dari kamu
    "berasa": "berasa"   # supaya aman
}

In [16]:
# normalisasi
normalisasi = pd.read_excel("kamus_normalisasi.xlsx")
normalisasi_dict = {}
for index, row in normalisasi.iterrows():
    normalisasi_dict[str(row.iloc[0])] = str(row.iloc[1])


def normalize_text(text):
    if not isinstance(text, str):
        return ""
    words = text.split()
    normalized = []
    for word in words:
        normalized.append(normalisasi_dict.get(word, word))
    return " ".join(normalized)

df["normalisasi"] = df["stripping"].apply(normalize_text)

In [17]:
# Tokenizing
df["tokenizing"] = df["normalisasi"].apply(lambda x: nltk.word_tokenize(re.sub(r"[^a-zA-Z0-9]", " ", x)))

In [18]:
# Stopword
df["stopword_removal"] = df["tokenizing"].apply(lambda x: [w for w in x if w not in stop_words])

In [19]:
# Stemming (Sastrawi)
df["stemming"] = df["stopword_removal"].apply(lambda x: [stemmer.stem(w) for w in x])

In [30]:
# Hapus huruf triple atau ebih
df["no_repeated"] = df["stemming"].astype(str).apply(
    lambda x: re.sub(r'(.)\1{2,}', r'\1', x))

In [31]:
df.head(10)

,nama_channel,judul_video,author,komentar,casefolding,stripping,normalisasi,tokenizing,stopword_removal,stemming,no_repeated
0,Pebbi Lieyanti,"BELI SEMUA MAKANAN VIRAL TIKTOK, PART 31!",@sugiartisunoto,akuu baru tauu kaka uda nikahh,akuu baru tauu kaka uda nikahh,akuu baru tauu kaka uda nikahh,akuu baru tauu kaka sudah nikahh,"[akuu, baru, tauu, kaka, sudah, nikahh]","[akuu, tauu, kaka, nikahh]","[akuu, tauu, kaka, nikahh]","['akuu', 'tauu', 'kaka', 'nikahh']"
1,Pebbi Lieyanti,"BELI SEMUA MAKANAN VIRAL TIKTOK, PART 31!",@Ayaww-channel,knp aga gelap ka,knp aga gelap ka,knp aga gelap ka,kenapa aga gelap ka,"[kenapa, aga, gelap, ka]","[aga, gelap, ka]","[aga, gelap, ka]","['aga', 'gelap', 'ka']"
2,Pebbi Lieyanti,"BELI SEMUA MAKANAN VIRAL TIKTOK, PART 31!",@Firasafira-o7p,"kaa masukan nih yaa\naudionya benahin ka, enta...","kaa masukan nih yaa\naudionya benahin ka, enta...","kaa masukan nih yaa\naudionya benahin ka, enta...","kaa masukan nan yaa audionya benahin ka, entah...","[kaa, masukan, nan, yaa, audionya, benahin, ka...","[kaa, masukan, nan, yaa, audionya, benahin, ka...","[kaa, masuk, nan, yaa, audio, benahin, ka, mic...","['kaa', 'masuk', 'nan', 'yaa', 'audio', 'benah..."
3,Pebbi Lieyanti,"BELI SEMUA MAKANAN VIRAL TIKTOK, PART 31!",@Flowerblue-h5e,Kak plis membeli mainan ❤🎉,kak plis membeli mainan ❤🎉,kak plis membeli mainan ❤🎉,kak tolong membeli mainan ❤🎉,"[kak, tolong, membeli, mainan]","[kak, tolong, membeli, mainan]","[kak, tolong, beli, main]","['kak', 'tolong', 'beli', 'main']"
4,Pebbi Lieyanti,"BELI SEMUA MAKANAN VIRAL TIKTOK, PART 31!",@TEGARGanteng-j7s,Kak pebbi makan makanan berhadiah dong 😅😅😅,kak pebbi makan makanan berhadiah dong 😅😅😅,kak pebbi makan makanan berhadiah dong 😅😅😅,kak pebbi makan makanan berhadiah nan 😅😅😅,"[kak, pebbi, makan, makanan, berhadiah, nan]","[kak, pebbi, makan, makanan, berhadiah, nan]","[kak, pebbi, makan, makan, hadiah, nan]","['kak', 'pebbi', 'makan', 'makan', 'hadiah', '..."
5,Pebbi Lieyanti,"BELI SEMUA MAKANAN VIRAL TIKTOK, PART 31!",@britneydexfine4853,gacorr,gacorr,gacorr,gacorr,[gacorr],[gacorr],[gacorr],['gacorr']
6,Pebbi Lieyanti,"BELI SEMUA MAKANAN VIRAL TIKTOK, PART 31!",@AlzeynAbhiChandra,Ci bikin konten make up lagi ci,ci bikin konten make up lagi ci,ci bikin konten make up lagi ci,ci bikin konten make up lagi ci,"[ci, bikin, konten, make, up, lagi, ci]","[ci, bikin, konten, make, up, ci]","[ci, bikin, konten, make, up, ci]","['ci', 'bikin', 'konten', 'make', 'up', 'ci']"
7,Pebbi Lieyanti,"BELI SEMUA MAKANAN VIRAL TIKTOK, PART 31!",@24_AmeliaSusanti,Kakkkk coba semua makanan khas balii gasiii ru...,kakkkk coba semua makanan khas balii gasiii ru...,kakkkk coba semua makanan khas balii gasiii ru...,kakkkk coba semua makanan khas balii gasiii ru...,"[kakkkk, coba, semua, makanan, khas, balii, ga...","[kakkkk, coba, makanan, khas, balii, gasiii, r...","[kakkkk, coba, makan, khas, balii, gasiii, ruj...","['kak', 'coba', 'makan', 'khas', 'balii', 'gas..."
8,Pebbi Lieyanti,"BELI SEMUA MAKANAN VIRAL TIKTOK, PART 31!",@matchaa432,3:41 aku udah pernah coba enak banget 🎉,3:41 aku udah pernah coba enak banget 🎉,3:41 aku udah pernah coba enak banget 🎉,3:41 aku sudah pernah coba enak banget 🎉,"[3, 41, aku, sudah, pernah, coba, enak, banget]","[3, 41, coba, enak, banget]","[3, 41, coba, enak, banget]","['3', '41', 'coba', 'enak', 'banget']"
9,Pebbi Lieyanti,"BELI SEMUA MAKANAN VIRAL TIKTOK, PART 31!",@Amle-w2mlg,Hmm nng gn gn😂😂❤❤❤,hmm nng gn gn😂😂❤❤❤,hmm nng gn gn😂😂❤❤❤,hmm nng gn gn😂😂❤❤❤,"[hmm, nng, gn, gn]","[hmm, nng, gn, gn]","[hmm, nng, gn, gn]","['hmm', 'nng', 'gn', 'gn']"


In [28]:
df.to_csv("data_komen_youtube.csv", index=False, encoding="utf-8-sig")

print("File tersimpan sebagai 'data_komen_youtube.csv'")

File tersimpan sebagai 'data_komen_youtube.csv'
